In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [62]:
df_init = pd.read_csv('clean_data.csv')

In [63]:
df_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16203 entries, 0 to 16202
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    16203 non-null  object 
 1   street                   16202 non-null  object 
 2   city                     16203 non-null  object 
 3   country_code             16195 non-null  object 
 4   address_text             16203 non-null  object 
 5   marker_icon              16203 non-null  object 
 6   workplace_type           16203 non-null  object 
 7   company_name             16203 non-null  object 
 8   company_url              16203 non-null  object 
 9   company_size             16203 non-null  object 
 10  experience_level         16203 non-null  object 
 11  latitude                 16203 non-null  float64
 12  longitude                16203 non-null  float64
 13  published_at             16203 non-null  object 
 14  remote_interview      

In [64]:
#removing rows with no information about salary
df_salary = df_init.dropna(subset=['from'])
df_salary.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10860 entries, 0 to 16202
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    10860 non-null  object 
 1   street                   10859 non-null  object 
 2   city                     10860 non-null  object 
 3   country_code             10857 non-null  object 
 4   address_text             10860 non-null  object 
 5   marker_icon              10860 non-null  object 
 6   workplace_type           10860 non-null  object 
 7   company_name             10860 non-null  object 
 8   company_url              10860 non-null  object 
 9   company_size             10860 non-null  object 
 10  experience_level         10860 non-null  object 
 11  latitude                 10860 non-null  float64
 12  longitude                10860 non-null  float64
 13  published_at             10860 non-null  object 
 14  remote_interview         10

In [66]:
df_skill_1 = df_salary.drop(columns = ['skill_2','skill_2_level','skill_3','skill_3_level'])
df_skill_2 = df_salary.drop(columns = ['skill_1','skill_1_level','skill_3','skill_3_level'])
df_skill_3 = df_salary.drop(columns = ['skill_1','skill_1_level','skill_2','skill_2_level'])
df_skill_1.sample()

,title,street,city,country_code,address_text,marker_icon,workplace_type,company_name,company_url,company_size,...,company_logo_url,remote,multilocation,way_of_apply,type,from,to,currency,skill_1,skill_1_level
5406,AWS DevOps Engineer,Centrum,Kraków,PL,"Centrum, Kraków",devops,remote,HL Tech,http://www.hltech.com,50-100,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Kraków', 'street': 'Centrum', 'slug...",redirect,b2b,19500.0,28500.0,pln,GitLab,4


In [70]:
df_skill_1.rename(columns={'skill_1': 'skill_name', 'skill_1_level': 'skill_level'}, inplace=True)
df_skill_2.rename(columns={'skill_2': 'skill_name', 'skill_2_level': 'skill_level'}, inplace=True)
df_skill_3.rename(columns={'skill_3': 'skill_name', 'skill_3_level': 'skill_level'}, inplace=True)

In [71]:
df_skills = pd.concat([df_skill_1, df_skill_2, df_skill_3])
df_skills

,title,street,city,country_code,address_text,marker_icon,workplace_type,company_name,company_url,company_size,...,company_logo_url,remote,multilocation,way_of_apply,type,from,to,currency,skill_name,skill_level
0,Junior DevOps Engineer,Aleja Murckowska 14,Katowice,PL,"Aleja Murckowska 14, Katowice",devops,remote,eSky.pl,https://www.esky.pl/o-nas/start,501-1000,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Wrocław', 'slug': 'esky-pl-junior-d...",redirect,b2b,6000.0,9000.0,pln,HTTP protocol,2.0
4,Jira Service Management Developer,Śródmieście,Warszawa,PL,"Śródmieście, Warszawa",admin,partly_remote,Link Group,http://linkgroup.pl/,200-250,...,https://bucket.justjoin.it/offers/company_logo...,False,"[{'city': 'Warszawa', 'street': 'Śródmieście',...",form,b2b,25000.0,28000.0,pln,JIRA,4.0
6,Project Manager,City Center,Warszawa,PL,"City Center, Warszawa",pm,partly_remote,emagine Polska,http://www.emagine.pl,1400+,...,https://bucket.justjoin.it/offers/company_logo...,False,"[{'city': 'Warszawa', 'street': 'City Center',...",form,b2b,19300.0,22500.0,pln,Confluence,5.0
8,Full Stack .NET Developer,Matejki 6,Gdańsk,PL,"Matejki 6, Gdańsk",net,remote,Infoprojekt,http://www.infoprojekt.pl,50,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Opole', 'slug': 'infoprojekt-full-s...",form,b2b,13000.0,18000.0,pln,SQL Server,3.0
12,Senior Backend Developer,Bednarska 1,Poznań,PL,"Bednarska 1, Poznań",javascript,remote,Bitnoise,https://bitnoise.pl,60,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Wrocław', 'slug': 'bitnoise-senior-...",redirect,b2b,19000.0,25000.0,pln,TypeScript,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16197,Senior Azure Devops Engineer (IaC),Centrum,Katowice,PL,"Centrum, Katowice",devops,remote,EPAM,https://careers.epam-poland.pl/,7000+,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Katowice', 'street': 'Centrum', 'sl...",redirect,permanent,14000.0,21000.0,pln,Terraform,4.0
16198,Senior Azure Devops Engineer (IaC),Centrum,Warszawa,PL,"Centrum, Warszawa",devops,remote,EPAM,https://careers.epam-poland.pl/,7000+,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Warszawa', 'street': 'Centrum', 'sl...",redirect,permanent,14000.0,21000.0,pln,Terraform,4.0
16199,Cloud Platform Developer,Centrum,Warszawa,PL,"Centrum, Warszawa",devops,remote,EPAM,https://careers.epam-poland.pl/,7000+,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Warszawa', 'street': 'Centrum', 'sl...",redirect,permanent,14000.0,21000.0,pln,GCP,3.0
16201,.NET Developer,-,Wrocław,PL,"-, Wrocław",net,remote,Nexio Management,http://nexio.pl/,500+,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Wrocław', 'street': '-', 'slug': 'n...",redirect,permanent,12000.0,16000.0,pln,.Net 4.5,4.0


In [76]:
df_skills.reset_index(drop=True, inplace = True)
df_skills

,title,street,city,country_code,address_text,marker_icon,workplace_type,company_name,company_url,company_size,...,company_logo_url,remote,multilocation,way_of_apply,type,from,to,currency,skill_name,skill_level
0,Junior DevOps Engineer,Aleja Murckowska 14,Katowice,PL,"Aleja Murckowska 14, Katowice",devops,remote,eSky.pl,https://www.esky.pl/o-nas/start,501-1000,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Wrocław', 'slug': 'esky-pl-junior-d...",redirect,b2b,6000.0,9000.0,pln,HTTP protocol,2.0
1,Jira Service Management Developer,Śródmieście,Warszawa,PL,"Śródmieście, Warszawa",admin,partly_remote,Link Group,http://linkgroup.pl/,200-250,...,https://bucket.justjoin.it/offers/company_logo...,False,"[{'city': 'Warszawa', 'street': 'Śródmieście',...",form,b2b,25000.0,28000.0,pln,JIRA,4.0
2,Project Manager,City Center,Warszawa,PL,"City Center, Warszawa",pm,partly_remote,emagine Polska,http://www.emagine.pl,1400+,...,https://bucket.justjoin.it/offers/company_logo...,False,"[{'city': 'Warszawa', 'street': 'City Center',...",form,b2b,19300.0,22500.0,pln,Confluence,5.0
3,Full Stack .NET Developer,Matejki 6,Gdańsk,PL,"Matejki 6, Gdańsk",net,remote,Infoprojekt,http://www.infoprojekt.pl,50,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Opole', 'slug': 'infoprojekt-full-s...",form,b2b,13000.0,18000.0,pln,SQL Server,3.0
4,Senior Backend Developer,Bednarska 1,Poznań,PL,"Bednarska 1, Poznań",javascript,remote,Bitnoise,https://bitnoise.pl,60,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Wrocław', 'slug': 'bitnoise-senior-...",redirect,b2b,19000.0,25000.0,pln,TypeScript,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32575,Senior Azure Devops Engineer (IaC),Centrum,Katowice,PL,"Centrum, Katowice",devops,remote,EPAM,https://careers.epam-poland.pl/,7000+,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Katowice', 'street': 'Centrum', 'sl...",redirect,permanent,14000.0,21000.0,pln,Terraform,4.0
32576,Senior Azure Devops Engineer (IaC),Centrum,Warszawa,PL,"Centrum, Warszawa",devops,remote,EPAM,https://careers.epam-poland.pl/,7000+,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Warszawa', 'street': 'Centrum', 'sl...",redirect,permanent,14000.0,21000.0,pln,Terraform,4.0
32577,Cloud Platform Developer,Centrum,Warszawa,PL,"Centrum, Warszawa",devops,remote,EPAM,https://careers.epam-poland.pl/,7000+,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Warszawa', 'street': 'Centrum', 'sl...",redirect,permanent,14000.0,21000.0,pln,GCP,3.0
32578,.NET Developer,-,Wrocław,PL,"-, Wrocław",net,remote,Nexio Management,http://nexio.pl/,500+,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Wrocław', 'street': '-', 'slug': 'n...",redirect,permanent,12000.0,16000.0,pln,.Net 4.5,4.0


In [77]:
most_common_skills = list(df_skills['skill_name'].value_counts().iloc[:20].index)
most_common_skills

['Java',
 'English',
 'Python',
 'JavaScript',
 'SQL',
 'React',
 'TypeScript',
 'AWS',
 'PHP',
 'Linux',
 'Spring',
 '.Net',
 'Node.js',
 'C#',
 'Docker',
 'Git',
 'Angular',
 'Agile',
 'Kubernetes',
 'JIRA']

In [78]:
df_most_common_skills=pd.DataFrame()

for row in range(len(df_skills)):
    if df_skills['skill_name'][row] in most_common_skills:
        df_most_common_skills= pd.concat([df_most_common_skills, df_skills.loc[[row]]])

df_most_common_skills

,title,street,city,country_code,address_text,marker_icon,workplace_type,company_name,company_url,company_size,...,company_logo_url,remote,multilocation,way_of_apply,type,from,to,currency,skill_name,skill_level
1,Jira Service Management Developer,Śródmieście,Warszawa,PL,"Śródmieście, Warszawa",admin,partly_remote,Link Group,http://linkgroup.pl/,200-250,...,https://bucket.justjoin.it/offers/company_logo...,False,"[{'city': 'Warszawa', 'street': 'Śródmieście',...",form,b2b,25000.0,28000.0,pln,JIRA,4.0
4,Senior Backend Developer,Bednarska 1,Poznań,PL,"Bednarska 1, Poznań",javascript,remote,Bitnoise,https://bitnoise.pl,60,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Wrocław', 'slug': 'bitnoise-senior-...",redirect,b2b,19000.0,25000.0,pln,TypeScript,3.0
8,Java Developer,Feliksa Nowowiejskiego 5,Poznań,PL,"Feliksa Nowowiejskiego 5, Poznań",java,remote,Apzumi,http://apzumi.com/,<45,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Poznań', 'street': 'Feliksa Nowowie...",redirect,b2b,12000.0,17000.0,pln,Spring,3.0
9,Kotlin/Java Engineer (Payment Gateway),Kopcińskiego 62,Łódź,PL,"Kopcińskiego 62, Łódź",java,remote,DNA Technology,https://dnatechnology.io/,40-50,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Warszawa', 'slug': 'dna-technology-...",redirect,b2b,22050.0,31920.0,pln,AWS,3.0
13,Senior Project Manager (Fixed-Price),"Zabłocie 43A, 30-701",Kraków,PL,"Zabłocie 43A, 30-701, Kraków",pm,remote,N-iX,https://careers.n-ix.com/,2000+,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Warszawa', 'slug': 'n-ix-senior-pro...",redirect,b2b,4200.0,4500.0,usd,Agile,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32547,Remote Senior .Net developer with Azure,Centrum,Gdańsk,PL,"Centrum, Gdańsk",net,remote,Kambu,https://www.kambu.pl/,20-30,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Gdańsk', 'street': 'Centrum', 'slug...",form,permanent,14000.0,19000.0,pln,.Net,4.0
32563,Tester oprogramowania,centrum,Kraków,PL,"centrum, Kraków",testing,remote,Currenda Sp. z o.o.,http://www.currenda.pl,230,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Kraków', 'street': 'centrum', 'slug...",redirect,permanent,7000.0,9000.0,pln,SQL,3.0
32564,Tester oprogramowania,centrum,Warszawa,PL,"centrum, Warszawa",testing,remote,Currenda Sp. z o.o.,http://www.currenda.pl,230,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Warszawa', 'street': 'centrum', 'sl...",redirect,permanent,7000.0,9000.0,pln,SQL,3.0
32565,Tester oprogramowania,centrum,Poznań,PL,"centrum, Poznań",testing,remote,Currenda Sp. z o.o.,http://www.currenda.pl,230,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Poznań', 'street': 'centrum', 'slug...",redirect,permanent,7000.0,9000.0,pln,SQL,3.0


In [79]:
df_most_common_skills.reset_index(inplace = True, drop = True)
df_most_common_skills

,title,street,city,country_code,address_text,marker_icon,workplace_type,company_name,company_url,company_size,...,company_logo_url,remote,multilocation,way_of_apply,type,from,to,currency,skill_name,skill_level
0,Jira Service Management Developer,Śródmieście,Warszawa,PL,"Śródmieście, Warszawa",admin,partly_remote,Link Group,http://linkgroup.pl/,200-250,...,https://bucket.justjoin.it/offers/company_logo...,False,"[{'city': 'Warszawa', 'street': 'Śródmieście',...",form,b2b,25000.0,28000.0,pln,JIRA,4.0
1,Senior Backend Developer,Bednarska 1,Poznań,PL,"Bednarska 1, Poznań",javascript,remote,Bitnoise,https://bitnoise.pl,60,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Wrocław', 'slug': 'bitnoise-senior-...",redirect,b2b,19000.0,25000.0,pln,TypeScript,3.0
2,Java Developer,Feliksa Nowowiejskiego 5,Poznań,PL,"Feliksa Nowowiejskiego 5, Poznań",java,remote,Apzumi,http://apzumi.com/,<45,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Poznań', 'street': 'Feliksa Nowowie...",redirect,b2b,12000.0,17000.0,pln,Spring,3.0
3,Kotlin/Java Engineer (Payment Gateway),Kopcińskiego 62,Łódź,PL,"Kopcińskiego 62, Łódź",java,remote,DNA Technology,https://dnatechnology.io/,40-50,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Warszawa', 'slug': 'dna-technology-...",redirect,b2b,22050.0,31920.0,pln,AWS,3.0
4,Senior Project Manager (Fixed-Price),"Zabłocie 43A, 30-701",Kraków,PL,"Zabłocie 43A, 30-701, Kraków",pm,remote,N-iX,https://careers.n-ix.com/,2000+,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Warszawa', 'slug': 'n-ix-senior-pro...",redirect,b2b,4200.0,4500.0,usd,Agile,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11494,Remote Senior .Net developer with Azure,Centrum,Gdańsk,PL,"Centrum, Gdańsk",net,remote,Kambu,https://www.kambu.pl/,20-30,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Gdańsk', 'street': 'Centrum', 'slug...",form,permanent,14000.0,19000.0,pln,.Net,4.0
11495,Tester oprogramowania,centrum,Kraków,PL,"centrum, Kraków",testing,remote,Currenda Sp. z o.o.,http://www.currenda.pl,230,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Kraków', 'street': 'centrum', 'slug...",redirect,permanent,7000.0,9000.0,pln,SQL,3.0
11496,Tester oprogramowania,centrum,Warszawa,PL,"centrum, Warszawa",testing,remote,Currenda Sp. z o.o.,http://www.currenda.pl,230,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Warszawa', 'street': 'centrum', 'sl...",redirect,permanent,7000.0,9000.0,pln,SQL,3.0
11497,Tester oprogramowania,centrum,Poznań,PL,"centrum, Poznań",testing,remote,Currenda Sp. z o.o.,http://www.currenda.pl,230,...,https://bucket.justjoin.it/offers/company_logo...,True,"[{'city': 'Poznań', 'street': 'centrum', 'slug...",redirect,permanent,7000.0,9000.0,pln,SQL,3.0


In [80]:
df_most_common_skills.to_csv('most_common_skills_with_salaries.csv', index=False)

In [81]:
df = df_most_common_skills.pivot(index = df_most_common_skills.columns.drop(['skill_name', 'skill_level']),
         columns = 'skill_name',
         values = 'skill_level').fillna(0).reset_index()

In [84]:
df.sample(10)

skill_name,title,street,city,country_code,address_text,marker_icon,workplace_type,company_name,company_url,company_size,...,JavaScript,Kubernetes,Linux,Node.js,PHP,Python,React,SQL,Spring,TypeScript
336,Architekt systemów IT,Centrum,Opole,PL,"Centrum, Opole",architecture,remote,XTB,https://www.xtb.com/pl,500+,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2322,IT Service Specialist,Puławska 182,Warszawa,PL,"Puławska 182, Warszawa",support,remote,7N,https://www.7n.com,800+,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3568,PHP Backend Developer,-,Kraków,PL,"-, Kraków",php,remote,PRINTU,https://printu.pl/,20+,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
3168,Mid .NET Fullstack Developer,-,Białystok,PL,"-, Białystok",net,remote,Software Interactive Sp. z o.o.,http://softint.pl,100+,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1857,Fullstack .NET Developer,Centrum,Wrocław,PL,"Centrum, Wrocław",net,remote,Smithing Systems,https://smithingsystems.com/pl,1-10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2633,Java Developer Cloud Native Stack,centrum,Bielsko-Biała,PL,"centrum, Bielsko-Biała",java,remote,Reply Polska,https://www.reply.com/en/about/careers/choose-...,"10,000",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5753,Senior Java + DevOps (GCP) Engineer,Remote,Wrocław,PL,"Remote, Wrocław",java,remote,JellyTech Sp. z o.o.,https://jellytech.com.pl,15-20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2222,IPC Developer,-,Lublin,PL,"-, Lublin",data,remote,7N,https://www.7n.com,800+,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
4551,Remote Java Backend Developer,Centrum,Szczecin,PL,"Centrum, Szczecin",java,remote,Kingfisher plc,https://careers.kingfisher.com,"77,000",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2383,JAVA & CAMUNDA DEVELOPER,-,Gdańsk,PL,"-, Gdańsk",java,remote,TUATARA,https://tuatara.pl/?utm_source=justjoin.it&utm...,>100,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [82]:
df.to_csv('skills_salaries_pivot_data.csv', index=False)